In [31]:
# coding: utf8

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import os
import re
import sys
from IPython.display import clear_output

In [32]:
# ### Variables

user_agent = {'User-agent': 'Mozilla/5.0'}

# ### Useful functions

# Base function to return HTML from an URL using BeautifulSoup
def download_soup(url):
    print('Starting request for: ' + url)
    response = requests.get(url, headers=user_agent)
    if response.status_code == 200:
        page = response.text
        soup = BeautifulSoup(page, 'lxml')
        print('Successful request!')
        return soup
    else:
        print('Unsuccessful request, status code: '+ response.status_code)

In [36]:
# ###Get rider info
# Parse throught a rider's html page to get:
# name, team, day of birth, country, height, weight
# and return it as a list

def get_uniquerider_infos(rider_url):
    print('Starting get_uniquerider_infos('+rider_url+')')

    rider_soup = download_soup("https://www.procyclingstats.com/rider/" + rider_url)
    rider_infos = pd.Series()
    
#     try:
#         fullname = rider_soup.find('title').text.encode('latin-1', 'ignore').decode('utf-8', 'ignore')
#     except(UnicodeEncodeError):
#         fullname = rider_soup.find('title').text
    rider_infos["fullname"] = rider_soup.find('title').text
    
#     try:
#         team = rider_soup.find('span', class_='red').text.encode('latin-1', 'ignore').decode('utf-8', 'ignore')
#     except(UnicodeEncodeError):
#         team = rider_soup.find('span', class_='red').text
    try:
        team = rider_soup.find('span', class_='red').text
    except:
        team = 'noteam'
    rider_infos["team"] = team    
        
    inter_soup = rider_soup.find('div', {"class":"rdr-info-cont"})
    list_birthdate = inter_soup.contents[1:4]
    try:
        birthdate_tmp = pd.datetime.strptime(list_birthdate[0] + list_birthdate[2][:-5], " %d %B %Y").date()
    except(TypeError):
        print(list_birthdate)
        birthdate_tmp = list_birthdate
    except:
        birthdate_tmp = pd.datetime.strptime(list_birthdate[0] + list_birthdate[2], " %d %B %Y").date()
    rider_infos["birthdate"] = birthdate_tmp
    #birthdate = int(birthdate_tmp.year*10000 + birthdate_tmp.month*100 + birthdate_tmp.day)
    
    try:
        country = inter_soup.find('a', class_='black').text.encode('latin-1', 'ignore').decode('utf-8', 'ignore')
    except(UnicodeEncodeError):
        country = inter_soup.find('a', class_='black').text
    rider_infos["country"] = country
    
    try:
        height = float(inter_soup.find(text='Height:').next.split()[0])
    except:
        height = 0
    rider_infos["height"] = height    
        
    try:
        weight = float(inter_soup.find(text='Weight:').next.split()[0])
    except:
        weight = 0
    rider_infos["weight"] = weight
    
    rider_infos["rider_url"] = rider_url
    
    print('Finished get_uniquerider_inf('+rider_url+')')
    return rider_infos

In [37]:
def get_allriders_info():
    print('Starting get_allriders_info')
    
    df_rider_urls = pd.read_csv("data/rider_urls.csv")
    n_riders = len(df_rider_urls)
    df_rider_infos = pd.DataFrame(columns=["fullname", "team", "birthdate", "country", "height", "weight", "rider_url"])
    
    for idx, row in df_rider_urls.iterrows():
        if idx < 0:
            pass
        else:
            # timer to not make too many requests too fast
            timer = 0.5 + 0.5 * random.random()
            time.sleep(timer)
            clear_output(wait=True)
            add = get_uniquerider_infos(row["rider_url"])
            print("Rider n° : " + str(idx) + " / " + str(n_riders), flush=True)
            df_rider_infos = df_rider_infos.append(add, ignore_index=True)   
            if idx % 100 == 0:
                df_rider_infos.to_csv("data/rider_infos.csv")
    
    df_rider_infos.to_csv("data/rider_infos.csv")
    
    return df_rider_infos

In [38]:
# get_uniquerider_infos("https://www.procyclingstats.com/rider/jan-andrej-cully")
output = get_allriders_info()

Starting get_uniquerider_infos(loic-vliegen)
Starting request for: https://www.procyclingstats.com/rider/loic-vliegen
Successful request!
Finished get_uniquerider_inf(loic-vliegen)
Rider n° : 7966 / 7967
